In [2]:
import os
import pyproj
import rasterio
import numpy as np
import geopandas as gpd

from tqdm import tqdm
from scipy import interpolate
from shapely.geometry import box, Polygon
from rasterio.merge import merge
from rasterio.windows import Window
from rasterio.enums import Resampling
from rasterio.transform import from_origin, from_bounds
from rasterio.features import geometry_mask

# normalized DSM Generation

In [9]:
def calculate_ndsm(dsm_path, dtm_path, output_path):
    # Open DSM and DTM raster files
    with rasterio.open(dsm_path) as dsm_src, rasterio.open(dtm_path) as dtm_src:
        # Read DSM and DTM data as NumPy arrays
        dsm = dsm_src.read(1)
        dtm = dtm_src.read(1)

        # Identify NoData areas in both DSM and DTM
        nodata_dsm = (dsm == dsm_src.nodata)
        nodata_dtm = (dtm == dtm_src.nodata)

        # Mask NoData values in both DSM and DTM
        dsm = np.ma.masked_array(dsm, mask=nodata_dsm)
        dtm = np.ma.masked_array(dtm, mask=nodata_dtm)

        # Calculate nDSM only where both DSM and DTM have valid data
        ndsm = dsm - dtm

        # Normalize nDSM by a scaling factor (e.g., 5.0)
        scaling_factor = 5.0
        ndsm_normalized = np.ma.masked_array(ndsm * scaling_factor, mask=(nodata_dsm | nodata_dtm))
        ndsm_normalized[ndsm_normalized < 0] = 0
        ndsm_normalized[ndsm_normalized > 255] = 255
        ndsm_normalized = ndsm_normalized.astype(np.uint8)
        
        # Write the nDSM to a new GeoTIFF file
        profile = dsm_src.profile.copy()
        profile.update(count=1, dtype=rasterio.uint8, nodata=None)

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(np.ma.filled(ndsm_normalized, fill_value=0), 1)

if __name__ == "__main__":
    # Replace with the actual file paths
    dsm_file_path = "~/dataset/img/italy/fe_dsm_merged_cog.tif"
    dtm_file_path = "~/dataset/img/italy/fe_dtm_merged_cog.tif"
    output_ndsm_path = "~/dataset/img/italy/fe_ndsm_merged.tif"

    calculate_ndsm(dsm_file_path, dtm_file_path, output_ndsm_path)


# merge nDSM with ortho images

In [8]:
def upsample_2d_array(input_array, target_resolution):
    """
    Upsample a 2D array to a given resolution using bilinear interpolation.

    Parameters:
    - input_array: 2D NumPy array to be upsampled.
    - target_resolution: Tuple (new_rows, new_columns) specifying the target resolution.

    Returns:
    - upsampled_array: 2D NumPy array after upsampling.
    """
    rows, cols = input_array.shape
    target_rows, target_cols = target_resolution

    # Create grids for the original and target resolutions
    x = np.arange(0, cols)
    y = np.arange(0, rows)
    x_new = np.linspace(0, cols - 1, target_cols)
    y_new = np.linspace(0, rows - 1, target_rows)

    # Create 2D interpolation function using bilinear interpolation
    interpolator = interpolate.RectBivariateSpline(y, x, input_array, kx=1, ky=1)

    # Perform interpolation on the new grids
    upsampled_array = interpolator(y_new, x_new).astype('uint8')

    return upsampled_array


def transform_bbox_epsg_22232_to_7791(bbox):
    # Define the source and destination projections
    transformer = pyproj.Transformer.from_crs("EPSG:22232", "EPSG:7791")

    # Transform the bounding box coordinates
    lon1, lat1, lon2, lat2 = bbox
    x1, y1 = transformer.transform(lon1, lat1)
    x2, y2 = transformer.transform(lon2, lat2)

    # Return the transformed bounding box coordinates
    transformed_bbox = [x1, y1, x2, y2]
    return transformed_bbox

def process_and_concatenate(input_folder, output_folder, merged_ndsm_path):
                                
    # List all 4-channel GeoTIFF files in the input folder
    tiff_files = [file for file in os.listdir(input_folder) if file.endswith(".tif")]
        
    for tiff_file in tqdm(tiff_files):
        tiff_path = os.path.join(input_folder, tiff_file)
        output_path = os.path.join(output_folder, f"{tiff_file}")

        # Get the spatial extent of the 4-channel GeoTIFF
        with rasterio.open(tiff_path) as tiff_src:
            bounds = tiff_src.bounds
            bounds = transform_bbox_epsg_22232_to_7791(bounds)
        
        with rasterio.open(merged_ndsm_path) as ndsm_src:
            # Calculate nDSM within the specified bounds        
            dst_window = rasterio.windows.from_bounds(*bounds, ndsm_src.transform)
            seg_ndsm = ndsm_src.read(window=dst_window, boundless=True, out_dtype='uint8', fill_value=0)
        
        seg_ndsm = upsample_2d_array(np.squeeze(seg_ndsm), (tiff_src.height, tiff_src.width))

        # Open the 4-channel GeoTIFF file
        with rasterio.open(tiff_path) as tiff_src:
            # Read the existing 4 channels
            channels = [tiff_src.read(i) for i in range(1, 5)]

            # Append the nDSM as the fifth channel
            channels.append(seg_ndsm)

            # Update the profile to include the fifth channel
            profile = tiff_src.profile.copy()
            profile.update(count=5, dtype=rasterio.uint8)

            # Write the new 5-channel GeoTIFF file
            with rasterio.open(output_path, 'w', **profile) as dst:
                for i, channel_data in enumerate(channels, start=1):
                    dst.write(channel_data, i)



input_folder = "~/dataset/img/italy/Ferrara_ortofoto2022_10_cm/"
output_folder = "~/dataset/img/italy/rgbin/"
merged_ndsm_path = "~/dataset/img/italy/fe_ndsm_merged.tif"

process_and_concatenate(input_folder, output_folder, merged_ndsm_path)


100%|██████████| 26/26 [56:26<00:00, 130.23s/it]


# Generate tiles

In [ ]:
def is_tile_inside_polygon(tile_transform, polygon, tile_size):
    tile_bounds = rasterio.transform.array_bounds(tile_size, tile_size, tile_transform)
    tile_geometry = box(*tile_bounds)
    return polygon.contains(tile_geometry)


def segment_and_save_tiles(folder_path, output_folder, shapefile_path):
    # List all the .tif files in the folder
    tif_files = [file for file in os.listdir(folder_path) if file.endswith(".tif")]
    
    # Read shapefile with boundary polygon
    gdf = gpd.read_file(shapefile_path)
    polygon = gdf.geometry.iloc[0]

    # Process each .tif file
    for tif_file in tqdm(tif_files):
        tif_file_path = os.path.join(folder_path, tif_file)

        # Read the original TIFF image
        with rasterio.open(tif_file_path) as src:
            image = src.read()
            transform = src.transform
            height, width = src.shape

        # Get image shape and calculate the number of tiles in both dimensions
        tile_size = 512
        nodata_value = 255
        num_tiles_height = height // tile_size
        num_tiles_width = width // tile_size

        # Loop through each tile
        for i in range(num_tiles_height):
            for j in range(num_tiles_width):
                # Extract the tile from the original image
                window = rasterio.windows.Window(j * tile_size, i * tile_size, tile_size, tile_size)
                tile = image[:, window.row_off:window.row_off + window.height, window.col_off:window.col_off + window.width]
                
                
                # Check if the tile is smaller than the desired size, if so, skip it
                if tile.shape[1] < tile_size or tile.shape[2] < tile_size:
                    continue
                
                nodata_pixels = np.sum(np.all(tile[:4, :, :] == nodata_value, axis=0))
                total_pixels = tile.shape[1] * tile.shape[2]

                # Check if the tile are mostly nodata area
                if nodata_pixels / total_pixels > 0.1:
                    continue
                
                # Calculate the new transform for the tile
                tile_transform = from_origin(
                transform.xoff + window.col_off * transform.a,
                transform.yoff + window.row_off * transform.e,
                transform.a,
                -transform.e
                )
                
                # Check if the tile is inside the valid ndsm
                if is_tile_inside_polygon(tile_transform, polygon, tile_size):
                    # Save the tile with a new name
                    tile_name = f"{os.path.splitext(os.path.basename(tif_file_path))[0]}_tile_{i * num_tiles_width + j}.tif"
                    tile_path = os.path.join(output_folder, tile_name)
                    with rasterio.open(
                        tile_path,
                        'w',
                        driver='GTiff',
                        height=tile_size,
                        width=tile_size,
                        count=5,
                        dtype='uint8',
                        crs=src.crs,
                        transform=tile_transform) as dst:
                        
                        dst.write(tile)

if __name__ == "__main__":
    # Specify the folder path containing the .tif files
    folder_path = "~/dataset/img/italy/rgbin/"
    # Specify the output folder for the segmented tiles
    output_folder = "~/dataset/pretrain_ready/italy/"
    # shp file of overlap area between DSM, DTM and ortho imagery
    # derived from QGIS 
    shapefile_path = '~/dataset/img/italy/italy_bounds.shp'

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Call the combined function
    segment_and_save_tiles(folder_path, output_folder, shapefile_path)
